In [ ]:
!pip install rasterio

In [ ]:
# For U-Net
!pip install git+https://github.com/qubvel/segmentation_models.pytorch
import segmentation_models_pytorch as smp

In [ ]:
import os
import shutil
import time
import random
import math
import re
import sys
import gc
import rasterio
import datetime
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from osgeo import gdal, ogr, osr

from tqdm import tqdm

In [ ]:
### For Google Colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
SEED_NUM = 0
NUM_SPLITS = 5 # Train in 1 epoch using image in (bach_size * NUM_SPLITS ** 2, 4, 3880 / NUM_SPLITS, 3880 / NUM_SPLITS)
SPL_WH = int(3880 / NUM_SPLITS)

ROOT_PATH = '/content/'
MAIN_DIRECTORY = ROOT_PATH + 'drive/MyDrive/Columbia/JPMC3-2' # Set the directory to save and load files
df = pd.read_csv(MAIN_DIRECTORY + '/training_set_naip_nlcd_both.csv')  # Save training_set_naip_nlcd_both.csv before running

# File path of the exported label (/catalog/labels/data/XXXX_hr_label-YYYY.geojson) from GroundWork,
# with XXXX of ID and YYYY of year like 3716_hr_label-2013.geojson
TEST_HR_DIRECTORY = MAIN_DIRECTORY + '/test_hr_geojson/'
TEST_ID_LIST = ['1950', '3137', '3313', '3716']

NUM_WE_SELECT = 512 # Roughly 60M * NUM_WE_SELECT size is required!! Max num = 4500, Max size = 270GB
EPOCH = 10
BATCH = 4

RENEW_TRAIN_DATASET_FLAG = True
RENEW_TEST_DATASET_FLAG = True
CODALAB_NAIP_DOWNLOAD_FLAG = True

# Set some, like ['FCN1l', 'FCN5l'], (or one, like ['FCN1l'], if running single model)
# out of 'FCN1l', 'FCN5l', 'U-NET18', and 'U-NET50'
MODEL_LIST = ['U-NET18', 'U-NET50'] # Do not change this list when training and evaluating ensemble models
OUTPUT_FILES_NAME = 'U-NET18_U-NET50'

# Set True when training ensemble models
TRAIN_ENSEMBLE_FLAG = True

# Set ID (0, 1, 2, 3, ...) of the ensemble model when training ensemble model with new initial network parameters and
# new random set of training dataset in mini-batch training.
ENSEMBLE_ID = 0

# Set True when evaluating ensemble model after training all ensemble models
EVALUATE_ENSEMBLE_FLAG = False
NUM_ENSEMBLE = 10

In [ ]:
if TRAIN_ENSEMBLE_FLAG:
    flag_check = (not TRAIN_ENSEMBLE_FLAG) or (not EVALUATE_ENSEMBLE_FLAG)
    if not flag_check:
        print('Do not train and evaluate at the same time. Evaluate after training ensemble models.')
        assert flag_check
    SEED_NUM = SEED_NUM + ENSEMBLE_ID
    OUTPUT_FILES_NAME = OUTPUT_FILES_NAME + '_ENS_' + str(ENSEMBLE_ID)

In [ ]:
# Not need to change these path unless you want to

# Directory to save numpy data and png files made from high reolution label geojson files
TEST_HR_NP_PNG_PATH = MAIN_DIRECTORY + '/data/test_hr_np_png/'

TRAIN_NAIP_DIRECTORY = MAIN_DIRECTORY + '/data/train_image/'
TRAIN_NLCD_DIRECTORY = MAIN_DIRECTORY + '/data/train_label/'
CHAIN_OUTPUT_DIRECTORY = MAIN_DIRECTORY + '/data/chain_label/'
TEST_NAIP_DIRECTORY = MAIN_DIRECTORY + '/data/test_image/'
TEST_NLCD_DIRECTORY = MAIN_DIRECTORY + '/data/test_label/'
TRAINED_OBJECT_DIRECTORY = MAIN_DIRECTORY + '/trained_objects/'
OUTPUT_IMAGE_DIRECTORY = MAIN_DIRECTORY + '/output_images/'
CODALAB_NAIP_DIRECTORY = MAIN_DIRECTORY + '/data/codalab_test_image/'
CODALAB_PRED_DIRECTORY = ROOT_PATH + 'codalab_test_prediction/'
CODALAB_SUBMISSION_DIRECTORY = ROOT_PATH + 'codalab_test_submission/'
CODALAB_SUBMISSION_ZIP_DIRECTORY = MAIN_DIRECTORY + '/codalab_submission/'

In [ ]:
NLCD_CLASSES = [0, 11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 52, 71, 81, 82, 90, 95] # 16 classes + 1 nodata class ("0").
NUM_DFC2021_CLASS = 5 # Output classes (0: Water, 1: Tree Canopy, 2: Low Vegetation , 3: Impervious, 4: None)
NLCD_IDX_TO_REDUCED_LC_MAP = np.array([
    4,#  0 No data 0
    0,#  1 Open Water
    4,#  2 Ice/Snow
    2,#  3 Developed Open Space
    3,#  4 Developed Low Intensity
    3,#  5 Developed Medium Intensity
    3,#  6 Developed High Intensity
    3,#  7 Barren Land
    1,#  8 Deciduous Forest
    1,#  9 Evergreen Forest
    1,# 10 Mixed Forest
    1,# 11 Shrub/Scrub
    2,# 12 Grassland/Herbaceous
    2,# 13 Pasture/Hay
    2,# 14 Cultivated Crops
    1,# 15 Woody Wetlands
    1,# 16 Emergent Herbaceious Wetlands
])

# set rescaling weights given to each class in loss function based on the distribution of target classes,
# (W, TC, LV, I) = (0.09, 0.453, 0.376, 0.081), and considering the percentage of None as 1/5 = 0.2
LOSS_WEIGHT = 1 / np.array([0.09, 0.453, 0.376, 0.081, 0.2]) / 5
MEAN_STD = {'2013': {'mean': [115.33700726, 128.85735351, 120.71228705, 157.53005651],
                     'std': [40.49209837, 39.70903589, 28.3181713, 67.99788979]},
            '2017': {'mean': [72.44318449, 86.37860107, 76.32562186, 130.50011478],
                     'std': [42.01297086, 35.15398457, 29.21830352, 59.32915633]}} # Pre-calculated by all training naip images

# Data Preparation

In [ ]:
def store_np_and_png(nlcd_path, hr_label_path, output_path, image_id_year):
    """
    Please export the high-res label(s) from GroundWork using "export data" tab.
    You only need to select the check-box of tile layer for the image options.
    Then you can DL catalog.zip from GroundWork.
    In this function, you will import the geojason file where high-res label data are stored.
    We also import the original NLCD low-res label, but this is only to get the size of the *label*.
    
    Note that we need to at least label the upper left and the lower right corner of the original NAIP image,
    because the largest and smallest coordinates are required to accurately transform to a png image file.

    Input:
        nlcd_path: File path of the downloaded NLCD image
        hr_label_path: File path of the exported label data (/catalog/labels/data/xxx.geojson) from GroundWork
        output_path: Folder path to save outputs
        image_id_year: 'Image_file_ID + "_" + year' to name outputs

    Output:
        image_gray_np: np.array of high-resolution label in gray scale
        image: rgb image with three channels (light blue: Water, dark green: Tree Canopy, light green: Low Vegetation, red: Impervious).
               Use this to visually check the curated high-res label
        image_gray: gray-scale image with only one channel (0: Water, 1: Tree Canopy, 2: Low Vegetation, 3: Impervious).
                Note that these are the same with Codalab https://codalab.lisn.upsaclay.fr/competitions/7908). Use this to test our models
    """
    from PIL import Image, ImageDraw
    import json

    def calc_map_LonLat(nlcd_path):
        def intermidiate_calc(extracted_xy):
            ms  = re.search('\..*"',  extracted_xy).group()[1:-1]
            sec = re.search('\'.*\.', extracted_xy).group()[1:-1]
            min = re.search('d.*\'',  extracted_xy).group()[1:-1]
            deg = re.search('.*d',    extracted_xy).group()[0:-1]
            return(int(deg) + int(min) / 60 + int(sec) / 3600 + int(ms) / 3600000)
        
        def calc_mapXY(extracted_UL_LR):
            mapX = - intermidiate_calc(re.search('\).*W', extracted_UL_LR).group()[3:-1])
            mapY =   intermidiate_calc(re.search('W,.*N', extracted_UL_LR).group()[2:-1])
            return(mapX, mapY)

        txt = gdal.Info(nlcd_path)
        extracted_UL = re.search('Upper Left.*\nLower Left', txt).group()
        extracted_LR = re.search('Lower Right.*\nCenter', txt).group()
        _mapXmin_, _mapYmax_ = calc_mapXY(extracted_UL)
        _mapXmax_, _mapYmin_ = calc_mapXY(extracted_LR)

        return(_mapXmin_, _mapYmax_, _mapXmax_, _mapYmin_)

    def make_save_image_np(nlcd_path, hr_label_path, output_path, image_id_year):
        # Open the data source and read in the extent
        data_source = ogr.Open(hr_label_path)
        if data_source is None:
            print ('Could not open file')
            sys.exit(1)

        mapXmin, mapXmax, mapYmin, mapYmax = data_source.GetLayer().GetExtent()
        _mapXmin_, _mapYmax_, _mapXmax_, _mapYmin_ = calc_map_LonLat(nlcd_path)

        print("X and Y-coordinates are longitude and latitude respectively",
              "\nmapXmin: ", mapXmin, "\nmapXmax: ", mapXmax, "\nmapYmin: ", mapYmin, "\nmapYmax: ", mapYmax,
              "\n\nfrom corresponding NLCD",
              "\nmapXmin: ", _mapXmin_, "\nmapXmax: ", _mapXmax_, "\nmapYmin: ", _mapYmin_, "\nmapYmax: ", _mapYmax_,
              "\nNote that it seems to be better to use the one from the exported high-res label from GroundWork",
              "\n\nW:", mapXmax - mapXmin, "\nH:", mapYmax - mapYmin)

        # Define pixel_size 
        # pixel_size = 0.5 # meters are one pixel
        # Create the target data source
        gdal_nlcd = gdal.Open(nlcd_path, gdal.GA_ReadOnly)
        target_Width = gdal_nlcd.RasterXSize
        target_Height = gdal_nlcd.RasterYSize

        pixel_size_x = abs(mapXmax - mapXmin) / target_Width
        pixel_size_y = abs(mapYmax - mapYmin) / target_Height
        print("pixel_size_x:", pixel_size_x, "\npixel_size_y:", pixel_size_y)

        image_TC = Image.new('RGB', (target_Width, target_Height))
        image_gray_TC = Image.new('L', (target_Width, target_Height))
        image_Im = Image.new('RGB', (target_Width, target_Height))
        image_gray_Im = Image.new('L', (target_Width, target_Height))
        image_W = Image.new('RGB', (target_Width, target_Height))
        image_gray_W = Image.new('L', (target_Width, target_Height))
        image_LV = Image.new('RGB', (target_Width, target_Height))
        image_gray_LV = Image.new('L', (target_Width, target_Height))

        draw_TC = ImageDraw.Draw(image_TC)
        draw_gray_TC = ImageDraw.Draw(image_gray_TC)
        draw_Im = ImageDraw.Draw(image_Im)
        draw_gray_Im = ImageDraw.Draw(image_gray_Im)
        draw_W = ImageDraw.Draw(image_W)
        draw_gray_W = ImageDraw.Draw(image_gray_W)
        draw_LV = ImageDraw.Draw(image_LV)
        draw_gray_LV = ImageDraw.Draw(image_gray_LV)

        # Loop through the features in the layer
        json_source = json.load(open(hr_label_path))
        for ftr in json_source.get('features'):
            att = ftr.get('properties')['default']

            for multipolygon in ftr['geometry']['coordinates']: #4D coordata
                for ply in multipolygon:
                    ply = np.array(ply)
                    loc = np.argmax(ply[:, 0])
                    v1 = ply[loc] - ply[loc - 1]
                    v2 = ply[loc + 1] - ply[loc - 1]

                    ply = (ply - [mapXmin, mapYmax]) * [1, -1] / [pixel_size_x, pixel_size_y]
                    ply = [(a[0], a[1]) for a in ply.tolist()]

                    if v1[0] * v2[1] - v1[1] * v2[0] >= 0:
                        if(att == 'Tree Canopy'):
                            color = (0, 128, 0); color2 = 1 #dark green
                            draw_TC.polygon(ply, fill = color, outline = None)
                            draw_gray_TC.polygon(ply, fill = color2, outline = None)
                        elif(att == 'Impervious'):
                            color = (255, 0, 0); color2 = 3 #red
                            draw_Im.polygon(ply, fill = color, outline = None)
                            draw_gray_Im.polygon(ply, fill = color2, outline = None)
                        elif(att == 'Water'):
                            color = (0, 225, 225); color2 = 0 #light blue
                            draw_W.polygon(ply, fill = color, outline = None)
                            draw_gray_W.polygon(ply, fill = color2, outline = None)
                        elif(att == 'Low Vegetation'):
                            color = (0, 255, 0); color2 = 2 #light green
                            draw_LV.polygon(ply, fill = color, outline = None)
                            draw_gray_LV.polygon(ply, fill = color2, outline = None)
                        else: raise ValueError('Wrong target class!')
                    else:
                        if(att == 'Tree Canopy'):
                            draw_TC.polygon(ply, fill = (0, 0, 0), outline = None)
                            draw_gray_TC.polygon(ply, fill = 0, outline = None)
                        elif(att == 'Impervious'):
                            draw_Im.polygon(ply, fill = (0, 0, 0), outline = None)
                            draw_gray_Im.polygon(ply, fill = 0, outline = None)
                        elif(att == 'Water'):
                            draw_W.polygon(ply, fill = (0, 0, 0), outline = None)
                            draw_gray_W.polygon(ply, fill = 0, outline = None)
                        elif(att == 'Low Vegetation'):
                            draw_LV.polygon(ply, fill = (0, 0, 0), outline = None)
                            draw_gray_LV.polygon(ply, fill = 0, outline = None)
                        else: raise ValueError('Wrong target class!')

        image_TC_np = np.array(image_TC)
        image_Im_np = np.array(image_Im)
        image_W_np = np.array(image_W)
        image_LV_np = np.array(image_LV)
        image_gray_TC_np = np.array(image_gray_TC)
        image_gray_Im_np = np.array(image_gray_Im)
        image_gray_W_np = np.array(image_gray_W)
        image_gray_LV_np = np.array(image_gray_LV)

        # prioritize in order of Im ->  LV ->  TC ->  W
        image_LV_np[np.sum(image_Im_np, axis = -1) > 0, :] = 0
        image_TC_np[np.sum(image_Im_np, axis = -1) > 0, :] = 0
        image_W_np[np.sum(image_Im_np, axis = -1) > 0, :] = 0
        image_gray_LV_np[image_gray_Im_np > 0] = 0
        image_gray_TC_np[image_gray_Im_np > 0] = 0
        image_gray_W_np[image_gray_Im_np > 0] = 0

        image_TC_np[np.sum(image_LV_np, axis = -1) > 0, :] = 0
        image_W_np[np.sum(image_LV_np, axis = -1) > 0, :] = 0
        image_gray_TC_np[image_gray_LV_np > 0] = 0
        image_gray_W_np[image_gray_LV_np > 0] = 0

        image_W_np[np.sum(image_TC_np, axis = -1) > 0, :] = 0
        image_gray_W_np[image_gray_TC_np > 0] = 0

        image_np = image_TC_np + image_Im_np + image_W_np + image_LV_np
        image_gray_np = image_gray_TC_np + image_gray_Im_np + image_gray_W_np + image_gray_LV_np
        np.save(output_path + "image_rgb-" + image_id_year, image_np)
        np.save(output_path + "image_gray-" + image_id_year, image_gray_np)

        return(image_np, image_gray_np)

    image_np, image_gray_np = make_save_image_np(nlcd_path, hr_label_path, output_path, image_id_year)
    image = Image.fromarray(image_np)
    image_gray = Image.fromarray(image_gray_np)

    print(f'\nsize of image: {image.size}', f'\nsize of image_gray: {image_gray.size}',
          f'\nsize of image -- rbg: {image_np.shape} as numpy array',
          f'\nsize of image_gray -- gray-scale: {image_gray_np.shape} as numpy array')
    print('\n---Distribution-------------------',
          '\n0: Water', '\n1: Tree Canopy', '\n2: Low Vegetation', '\n3: Impervious\n\n',
          pd.Series(image_gray_np.flatten()).value_counts())

    gdal_nlcd = gdal.Open(nlcd_path, gdal.GA_ReadOnly)
    nlcd_np = np.array([gdal_nlcd.GetRasterBand(i + 1).ReadAsArray() for i in range(gdal_nlcd.RasterCount)])

    fig, ax = plt.subplots(1, 3, figsize = (15, 5))
    ax[0].imshow(image)
    ax[0].set_title("high-res label (rgb)")
    ax[1].imshow(image_gray, cmap = 'gray')
    ax[1].set_title("high-res label (gray-scale)")
    ax[2].imshow(nlcd_np[0], cmap = 'gray')
    ax[2].set_title("NLCD low-res label (gray-scale)")
    plt.show()

    image.save(output_path + "image_rgb-" + image_id_year + '.png')
    image_gray.save(output_path + "image_gray-" + image_id_year + '.png')

In [ ]:
import urllib
def download_file(save_path, save_file_name, url):
    os.makedirs(save_path, exist_ok = True)
    try:
        with urllib.request.urlopen(url) as web_file, open(save_path + save_file_name, 'wb') as local_file:
            local_file.write(web_file.read())
    except urllib.error.URLError as e:
        print(e)

In [ ]:
if RENEW_TEST_DATASET_FLAG:
    file_names = os.listdir(TEST_HR_DIRECTORY)
    shutil.rmtree(TEST_NAIP_DIRECTORY, ignore_errors = True)
    shutil.rmtree(TEST_NLCD_DIRECTORY, ignore_errors = True)
    shutil.rmtree(TEST_HR_NP_PNG_PATH, ignore_errors = True)
    os.makedirs(TEST_HR_NP_PNG_PATH, exist_ok = True)

    for fn in sorted([file for file in file_names if file.endswith('.geojson')]):
        print(fn)
        id = fn[:4]
        year = fn[-12:-8]

        naip_file_name = id + '_naip-' + year + '.tif'
        download_file(save_path = TEST_NAIP_DIRECTORY, save_file_name = naip_file_name,
                      url = 'https://dfc2021.blob.core.windows.net/competition-data/naip-' + year + '/' + naip_file_name)

        if year == '2017':
            nlcd_year = '2016'
        else:
            nlcd_year = year
        nlcd_file_name = id + '_nlcd-' + nlcd_year + '.tif'
        download_file(save_path = TEST_NLCD_DIRECTORY, save_file_name = nlcd_file_name,
                      url = 'https://dfc2021.blob.core.windows.net/competition-data/nlcd-' + nlcd_year + '/' + nlcd_file_name)
        
        store_np_and_png(nlcd_path = TEST_NLCD_DIRECTORY + nlcd_file_name, hr_label_path = TEST_HR_DIRECTORY + fn,
                         output_path = TEST_HR_NP_PNG_PATH, image_id_year = id + '_' + year)

In [ ]:
if RENEW_TRAIN_DATASET_FLAG:
    if EVALUATE_ENSEMBLE_FLAG:
        print('Do not need to renew train dataset when evaluating ensemble models')
        assert not EVALUATE_ENSEMBLE_FLAG

    random.seed(SEED_NUM)

    shutil.rmtree(TRAIN_NAIP_DIRECTORY, ignore_errors = True)
    shutil.rmtree(TRAIN_NLCD_DIRECTORY, ignore_errors = True)

    ris_dl = random.sample(range(len(df)), k = NUM_WE_SELECT)
    df_dl = df.iloc[ris_dl]

    start_time = time.time()
    for _, url in tqdm(enumerate(df_dl['image_fn'])):
        download_file(save_path = TRAIN_NAIP_DIRECTORY, save_file_name = url[65:], url = url)
    for _, url in tqdm(enumerate(df_dl['label_fn'])):
        download_file(save_path = TRAIN_NLCD_DIRECTORY, save_file_name = url[65:], url = url)

    time_lapsed = time.time() - start_time
    print(f'\n{time_lapsed // 60} min {int(time_lapsed % 60)} sec lapsed')

# Train model

## Define Functions

In [ ]:
class FCN1l(nn.Module):
    # Simplest FCN model reducing to multinomial Logistic regression model
    # Input: 4D tensor (batch_size, num_input_channels, input_size, input_size)

    def __init__(self, num_input_channels, num_output_classes, kernel_size = 3, padding = 1):
        super(FCN1l, self).__init__()

        self.conv1 = nn.Conv2d(num_input_channels, num_output_classes, kernel_size = kernel_size, stride = 1, padding = padding)

    def forward(self, inputs):
        x = self.conv1(inputs)

        return x  # Output: 4D tensor (batch_size, num_output_classes, input_size, input_size)

class FCN5l(nn.Module):
    # FCN model with 5 layers
    # Input: 4D tensor (batch_size, num_input_channels, input_size, input_size)

    def __init__(self, num_input_channels, num_output_classes, kernel_size = 3, padding = 1, num_filters = 64):
        super(FCN5l, self).__init__()

        self.layers = nn.Sequential(
            nn.Conv2d(num_input_channels, num_filters, kernel_size = kernel_size, stride = 1, padding = padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters,        num_filters, kernel_size = kernel_size, stride = 1, padding = padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters,        num_filters, kernel_size = kernel_size, stride = 1, padding = padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters,        num_filters, kernel_size = kernel_size, stride = 1, padding = padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters,        num_filters, kernel_size = kernel_size, stride = 1, padding = padding),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(),
            nn.Conv2d(num_filters, num_output_classes, kernel_size = 1          , stride = 1, padding = 0)
        )
        
    def forward(self, inputs):
        return self.layers(inputs)  # Output: 4D tensor (batch_size, num_output_classes, input_size, input_size)

In [ ]:
def padding(image_np, rgb_or_gray = "rgb", image_or_label = "image"):
    if (image_or_label == "image") and (rgb_or_gray == "rgb"):
        width, height = image_np.shape[1], image_np.shape[2]
    else:
        width, height = image_np.shape[0], image_np.shape[1]

    pw_x_bef = int((3880 - width) / 2)
    pw_x_aft = 3880 - width - pw_x_bef
    pw_y_bef = int((3880 - height) / 2)
    pw_y_aft = 3880 - height - pw_y_bef
    
    if rgb_or_gray == "rgb":
        if image_or_label == "image":
            pad_width = ((0, 0), (pw_x_bef, pw_x_aft), (pw_y_bef, pw_y_aft))
        elif image_or_label == "label":
            pad_width = ((pw_x_bef, pw_x_aft), (pw_y_bef, pw_y_aft), (0, 0))
        else:
            print ('Specify image_or_label by "image" or "label"')
            sys.exit(1)

        image_np = np.pad(image_np, pad_width,
                          constant_values = ((0, 0), (0, 0), (0, 0)))
    elif rgb_or_gray == "gray": # if rgb_or_gray == "gray", image_or_label = "label"
        image_np = np.pad(image_np, ((pw_x_bef, pw_x_aft), (pw_y_bef, pw_y_aft)),
                          constant_values = ((4, 4), (4, 4)))
    else:
        print ('Specify rgb_or_gray by "rgb" or "gray"')
        sys.exit(1)
    
    return(image_np, pw_x_bef, pw_x_aft, pw_y_bef, pw_y_aft)

def make_image_data(image_path, image_loader):
    image_np_s = []
    image_np_org_s = []
    pad_width = {'pw_x_bef': [], 'pw_x_aft': [], 'pw_y_bef': [], 'pw_y_aft': []}
    for file in image_loader:
        ds = gdal.Open(os.path.join(image_path, file), gdal.GA_ReadOnly)
        image_np = np.array([ds.GetRasterBand(i + 1).ReadAsArray() for i in range(ds.RasterCount)])
        image_np, pw_x_bef, pw_x_aft, pw_y_bef, pw_y_aft = padding(image_np, rgb_or_gray = "rgb", image_or_label = "image")

        image_np_org_s.append(image_np)
        year = file[-8:-4]
        image_np = (image_np - np.array(MEAN_STD[year]['mean']).reshape(4,1,1)) / \
                        np.array(MEAN_STD[year]['std']).reshape(4,1,1)
        pad_width['pw_x_bef'].append(pw_x_bef)
        pad_width['pw_x_aft'].append(pw_x_aft)
        pad_width['pw_y_bef'].append(pw_y_bef)
        pad_width['pw_y_aft'].append(pw_y_aft)

        for i in range(NUM_SPLITS):
            for ii in range(NUM_SPLITS):
                image_np_s.append(image_np[:, i * SPL_WH:(i + 1) * SPL_WH, ii * SPL_WH:(ii + 1) * SPL_WH])            

    image_np_org_s = np.array(image_np_org_s)
    image_np_s = np.array(image_np_s)
    assert image_np_s.shape == (len(image_loader) * NUM_SPLITS ** 2, 4, SPL_WH, SPL_WH)

    image_data = torch.from_numpy(image_np_s).float().to(device)
    return(image_data, image_np_s, image_np_org_s, pad_width)

def make_test_targets(label_path):
    test_label_np_s = []
    test_label_loader = sorted([file for file in os.listdir(label_path) if file.endswith('.npy') and (file.find("gray") > -1)])
    for file in test_label_loader:
        test_label_np = np.load(os.path.join(label_path, file), gdal.GA_ReadOnly)
        test_label_np, _, _, _, _ = padding(test_label_np, rgb_or_gray = "gray", image_or_label = "label")
        for i in range(NUM_SPLITS):
            for ii in range(NUM_SPLITS):
                test_label_np_s.append(test_label_np[i * SPL_WH:(i + 1) * SPL_WH, ii * SPL_WH:(ii + 1) * SPL_WH])

    test_label_np_s = np.array(test_label_np_s)
    assert test_label_np_s.shape == (len(test_label_loader) * NUM_SPLITS ** 2, SPL_WH, SPL_WH)

    test_targets = torch.from_numpy(test_label_np_s).type(torch.LongTensor).to(device)
    return(test_targets, test_label_np_s)

def calc_IoU(pred_label, g_truth_label, calc_label):
    isect = torch.sum(10 * pred_label + g_truth_label == 11 * calc_label)
    union = torch.sum(pred_label == calc_label) + torch.sum(g_truth_label == calc_label) - isect

    return(isect / union)
    
def reshape_data(data):
    num_data = int(data.shape[0] / NUM_SPLITS ** 2)
    if data.ndim == 4:
        reshaped_data = np.empty((0, 4, 3880, 3880))
        for n in range(num_data):
            reshaped_data_r = np.empty((1, 4, 0, 3880))
            for i in range(NUM_SPLITS):
                reshaped_data_c = np.empty((1, 4, SPL_WH, 0))
                for ii in range(NUM_SPLITS):
                    reshaped_data_c = np.append(reshaped_data_c, data[[n * NUM_SPLITS ** 2 + i * NUM_SPLITS + ii], :, :, :], 3)
                reshaped_data_r = np.append(reshaped_data_r, reshaped_data_c, 2)
            reshaped_data = np.append(reshaped_data, reshaped_data_r, 0)
    else:
        reshaped_data = np.empty((0, 3880, 3880))
        for n in range(num_data):
            reshaped_data_r = np.empty((1, 0, 3880))
            for i in range(NUM_SPLITS):
                reshaped_data_c = np.empty((1, SPL_WH, 0))
                for ii in range(NUM_SPLITS):
                    reshaped_data_c = np.append(reshaped_data_c, data[[n * NUM_SPLITS ** 2 + i * NUM_SPLITS + ii], :, :], 2)
                reshaped_data_r = np.append(reshaped_data_r, reshaped_data_c, 1)
            reshaped_data = np.append(reshaped_data, reshaped_data_r, 0)

    return(reshaped_data)

In [ ]:
def training(model, optimizer, epoch, bsize, ite_show_log = 32, chain_flag = False, chain_num = 1,
             losses = [], test_losses = [], l_IoU = [[]], l_test_IoU = [[]]):    
    def make_train_targets(label_path, ris, chain_flag = False, chain_num = 1):
        label_np_s = []
        label_loader = sorted([file for file in os.listdir(label_path) if file.endswith('.tif')])
        for file in [label_loader[ri] for ri in ris]:
            ds = gdal.Open(os.path.join(label_path, file), gdal.GA_ReadOnly)
            label_np = np.array([ds.GetRasterBand(i + 1).ReadAsArray() for i in range(ds.RasterCount)])
            label_np = np.squeeze(label_np)
            if not(chain_flag and (chain_num > 1)):
                for i, ele in enumerate(NLCD_CLASSES):
                    label_np = np.where(label_np == ele, NLCD_IDX_TO_REDUCED_LC_MAP[i], label_np)
            label_np, _, _, _, _ = padding(label_np, rgb_or_gray = "gray", image_or_label = "label")
            
            for i in range(NUM_SPLITS):
                for ii in range(NUM_SPLITS):
                    label_np_s.append(label_np[i * SPL_WH:(i + 1) * SPL_WH, ii * SPL_WH:(ii + 1) * SPL_WH].tolist())

        label_np_s = np.array(label_np_s)
        assert label_np_s.shape == (len(ris) * NUM_SPLITS ** 2, SPL_WH, SPL_WH)

        train_targets = torch.from_numpy(label_np_s).type(torch.LongTensor).to(device)
        return(train_targets)


    start_time = time.time()
    !nvidia-smi
    criterion = nn.CrossEntropyLoss(weight = torch.tensor(LOSS_WEIGHT).type(torch.cuda.FloatTensor).to(device))
    for i in range(NUM_DFC2021_CLASS - len(l_IoU)): l_IoU.append([]), l_test_IoU.append([])

    epo1_ite_num = int(NUM_WE_SELECT / bsize)
    for ite in tqdm(range(epoch * epo1_ite_num)):
        ## Train
        ris = random.sample(range(NUM_WE_SELECT), k = bsize)
        image_loader = sorted([file for file in os.listdir(TRAIN_NAIP_DIRECTORY) if file.endswith('.tif')])
        image_loader = [image_loader[ri] for ri in ris]
        train_data, _, _, _ = make_image_data(image_path = TRAIN_NAIP_DIRECTORY, image_loader = image_loader)

        if chain_flag and (chain_num > 1):
            chain_prev_n_directory = CHAIN_OUTPUT_DIRECTORY + OUTPUT_FILES_NAME + '_model_' + str(chain_num - 1) + '/'
            train_targets = make_train_targets(label_path = chain_prev_n_directory, ris = ris,
                                               chain_flag = chain_flag, chain_num = chain_num)
        else:
            train_targets = make_train_targets(label_path = TRAIN_NLCD_DIRECTORY, ris = ris,
                                               chain_flag = chain_flag, chain_num = chain_num)

        preds_s = np.empty((0, SPL_WH, SPL_WH))
        for i in range(int(train_data.shape[0] / bsize)):
            optimizer.zero_grad()
            outputs = model(train_data[i * bsize:(i + 1) * bsize])
            preds = torch.max(outputs, dim = 1).indices
            preds_s = np.append(preds_s, preds.to('cpu').detach().numpy().copy(), 0)
            loss = criterion(outputs, train_targets[i * bsize:(i + 1) * bsize])
            loss.backward()

            optimizer.step()

        losses.append(loss.item())
        if train_data.shape[0] % bsize > 0:
            optimizer.zero_grad()
            outputs = model(train_data[(i + 1) * bsize:])
            preds = torch.max(outputs, dim = 1).indices
            preds_s = np.append(preds_s, preds.to('cpu').detach().numpy().copy(), 0)
            loss = criterion(outputs, train_targets[(i + 1) * bsize:])
            loss.backward()

            optimizer.step()   

        preds_s = torch.tensor(preds_s).to(device)
        for i in range(NUM_DFC2021_CLASS): l_IoU[i].append(calc_IoU(preds_s, train_targets, i).item())
 
        if ite == 0:
            print("\nmemory_usage_after_train")
            print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
            print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
            print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

        ## Test
        if ite == 0:
            test_image_loader = sorted([file for file in os.listdir(TEST_NAIP_DIRECTORY) if file.endswith('.tif')])
            test_data, test_image_np_s, test_image_np_org_s, _ = make_image_data(image_path = TEST_NAIP_DIRECTORY, image_loader = test_image_loader)
            test_targets, test_label_np_s = make_test_targets(label_path = TEST_HR_NP_PNG_PATH)

        test_preds_s = np.empty((0, SPL_WH, SPL_WH))
        for i in range(int(test_data.shape[0] / bsize)):
            test_outputs = model(test_data[i * bsize:(i + 1) * bsize])
            test_preds = torch.max(test_outputs, dim = 1).indices
            test_preds_s = np.append(test_preds_s, test_preds.to('cpu').detach().numpy().copy(), 0)
            test_loss = criterion(test_outputs, test_targets[i * bsize:(i + 1) * bsize])
        
        test_losses.append(test_loss.item())
        if test_data.shape[0] % bsize > 0:
            test_outputs = model(test_data[(i + 1) * bsize:])
            test_preds = torch.max(test_outputs, dim = 1).indices
            test_preds_s = np.append(test_preds_s, test_preds.to('cpu').detach().numpy().copy(), 0)

        test_preds_s = torch.tensor(test_preds_s).to(device)
        for i in range(NUM_DFC2021_CLASS): l_test_IoU[i].append(calc_IoU(test_preds_s, test_targets, i).item())
        if ite == 0:
            print("\nmemory_usage_after_test")
            print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
            print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
            print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

        time_lapsed = time.time() - start_time
        if ite % ite_show_log == 0:
            print(f'\n{int(time_lapsed // 60)} min {int(time_lapsed % 60)} sec lapsed, \n \
                    {ite} steps: Loss = {losses[-1]:.4f}, Test Loss = {test_losses[-1]:.4f}, \n \
                    IoU (Water) = {l_IoU[0][-1]:.4f}, IoU (Tree Canopy) = {l_IoU[1][-1]:.4f}, \n \
                    IoU (Low Vegetation) = {l_IoU[2][-1]:.4f}, IoU (Impervious) = {l_IoU[3][-1]:.4f}, \n \
                    Test IoU (Water) = {l_test_IoU[0][-1]:.4f}, Test IoU (Tree Canopy) = {l_test_IoU[1][-1]:.4f}, \n \
                    Test IoU (Low Vegetation) = {l_test_IoU[2][-1]:.4f}, Test IoU (Impervious) = {l_test_IoU[3][-1]:.4f}'
                    )
        if ((ite + 1) % epo1_ite_num == 0):# and int((ite + 1) / epo1_ite_num) > 4:
            print('\naverage test losses in this epoch:', np.mean(test_losses[-epo1_ite_num:]))
            print('rate of change of average test losses in this epoch from the previous one:',
                  (np.mean(test_losses[-epo1_ite_num * 2:-epo1_ite_num]) - np.mean(test_losses[-epo1_ite_num:])) / np.mean(test_losses[-epo1_ite_num:]))
            
    
    test_preds_s = torch.from_numpy(reshape_data(test_preds_s.to('cpu').detach().numpy().copy())).to(device)
    test_label_np_s = torch.from_numpy(reshape_data(test_label_np_s)).to(device)

    print(f'\n{time_lapsed // 60} min {int(time_lapsed % 60)} sec lapsed')
    return(model, losses, test_losses, l_IoU, l_test_IoU, test_preds_s, test_image_np_s, test_image_np_org_s, test_label_np_s)

In [ ]:
def reshape_and_save_tif(preds, pw_x_bef, pw_x_aft, pw_y_bef, pw_y_aft, file, naip_directory, save_directory):
    if pw_x_aft > 0:
        preds = preds[pw_x_bef:-pw_x_aft, :]
    else:
        preds = preds[pw_x_bef:, :]
    if pw_y_aft > 0:
        preds = preds[:, pw_y_bef:-pw_y_aft]
    else:
        preds = preds[:, pw_y_bef:]
        
    with rasterio.open(os.path.join(naip_directory, file)) as f:
        input_profile = f.profile.copy()

    output_profile = input_profile.copy()
    output_profile["driver"] = "GTiff"
    output_profile["dtype"] = "uint8"
    output_profile["count"] = 1
    output_profile["nodata"] = 0

    output_fn = file.replace("naip", "predictions")
    output_fn = os.path.join(save_directory, output_fn)
    with rasterio.open(output_fn, "w", **output_profile) as f:
        f.write(preds, 1)

def make_save_preds(model, bsize, image_loader, naip_dir, save_pred_dir, codalab_flag):
    def batch_make_save(model, bsize, image_loader, naip_dir, save_pred_dir, codalab_flag):
        data, _, _, pad_width = make_image_data(image_path = naip_dir, image_loader = image_loader)

        preds_s = np.empty((0, SPL_WH, SPL_WH))
        for i in range(int(data.shape[0] / bsize)):
            outputs = model(data[i * bsize:(i + 1) * bsize])
            preds = torch.max(outputs, dim = 1).indices
            preds_s = np.append(preds_s, preds.to('cpu').detach().numpy().copy(), 0)

        if data.shape[0] % bsize > 0:
            outputs = model(data[(i + 1) * bsize:])
            preds = torch.max(outputs, dim = 1).indices
            preds_s = np.append(preds_s, preds.to('cpu').detach().numpy().copy(), 0)

        reshaped_preds_s = reshape_data(preds_s).reshape([-1, 3880, 3880])
        num_data = len(image_loader)
        assert reshaped_preds_s.shape == (num_data, 3880, 3880)

        if codalab_flag:
            # Class "None" -> modified to "Tree Canopy" (because TC is most prevalent)
            reshaped_preds_s[reshaped_preds_s == 4] = 1

        for n in range(num_data):
            reshape_and_save_tif(preds = reshaped_preds_s[n], pw_x_bef = pad_width['pw_x_bef'][n],
                                 pw_x_aft = pad_width['pw_x_aft'][n], pw_y_bef = pad_width['pw_y_bef'][n],
                                 pw_y_aft = pad_width['pw_y_aft'][n], file = image_loader[n],
                                 naip_directory = naip_dir, save_directory = save_pred_dir)


    shutil.rmtree(save_pred_dir, ignore_errors = True)
    os.makedirs(save_pred_dir, exist_ok = True)

    start_time = time.time()
    for ite in tqdm(range(int(len(image_loader) / bsize))):
        batch_make_save(model = model, bsize = bsize, image_loader = image_loader[(ite * bsize):((ite + 1) * bsize)],
                        naip_dir = naip_dir, save_pred_dir = save_pred_dir, codalab_flag = codalab_flag)
    if len(image_loader) % bsize > 0:
        batch_make_save(model = model, bsize = bsize, image_loader = image_loader[((ite + 1) * bsize):],
                        naip_dir = naip_dir, save_pred_dir = save_pred_dir, codalab_flag = codalab_flag)
        
    print("\nmemory_usage_after_making_preds")
    print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

    time_lapsed = time.time() - start_time
    print(f'\n{time_lapsed // 60} min {int(time_lapsed % 60)} sec lapsed')

In [ ]:
def make_rgb_label(label_np):
    """
    0: Water, 1: Tree Canopy, 2: Low Vegetation, 3: Impervious
    light blue: Water, dark green: Tree Canopy, light green: Low Vegetation, red: Impervious
    """
    label_np_rgb = np.zeros((len(label_np), 3880, 3880, 3), dtype = np.uint8)
    for i in range(len(label_np)):
        label_np_rgb[i][label_np[i] == 0, :] = [0, 225, 225]
        label_np_rgb[i][label_np[i] == 1, :] = [0, 128, 0]
        label_np_rgb[i][label_np[i] == 2, :] = [0, 225, 0]
        label_np_rgb[i][label_np[i] == 3, :] = [255, 0, 0]

    return(label_np_rgb)

def visualize_save_pred(test_preds, test_image_np_org_s, chain_flag, chain_num,
                        uncertainty_s = [], ex_flag = False):
    titles = []
    test_label_loader_rgb = sorted([file for file in os.listdir(TEST_HR_NP_PNG_PATH)
                                    if file.endswith('.npy') and (file.find("rgb") > -1)])
    for fn in test_label_loader_rgb: titles.append(re.search('-.*\.', fn).group()[1:-1])
    num_file = len(test_label_loader_rgb)
    
    hr_label_s_rgb = []
    for file in test_label_loader_rgb:
        if file.endswith('.npy'):
            test_label_np_rgb = np.load(os.path.join(TEST_HR_NP_PNG_PATH, file), gdal.GA_ReadOnly)
            padded_hr_label_rgb, _, _, _, _ = padding(test_label_np_rgb, rgb_or_gray = "rgb", image_or_label = "label")
            hr_label_s_rgb.append(padded_hr_label_rgb)

    if test_preds.dtype == 'float64':
        test_preds_np_rgb = make_rgb_label(test_preds)
    else:
        test_preds_np_rgb = make_rgb_label(test_preds.to('cpu').detach().numpy().copy())    

    nlcd_np_s = []
    test_nlcd_loader = sorted([file for file in os.listdir(TEST_NLCD_DIRECTORY) if file.endswith('.tif')])
    for file in test_nlcd_loader:
        ds = gdal.Open(os.path.join(TEST_NLCD_DIRECTORY, file), gdal.GA_ReadOnly)
        label_np = np.array([ds.GetRasterBand(i + 1).ReadAsArray() for i in range(ds.RasterCount)])
        label_np = np.squeeze(label_np)
        for i, ele in enumerate(NLCD_CLASSES):
            label_np = np.where(label_np == ele, NLCD_IDX_TO_REDUCED_LC_MAP[i], label_np)

        padded_ncld_np, _, _, _, _ = padding(label_np, rgb_or_gray = "gray", image_or_label = "label")
        nlcd_np_s.append(padded_ncld_np)

    nlcd_np_s_rgb = make_rgb_label(nlcd_np_s)
    test_image_np_org_s = test_image_np_org_s.astype(np.uint8)

    print(titles)
    fig, ax = plt.subplots(num_file, 4, figsize = (20, 10 * int(num_file / 2)))
    ax[0, 0].set_title('NAIP image')
    ax[0, 1].set_title('Ground truth')
    ax[0, 2].set_title('Prediction')
    ax[0, 3].set_title('NLCD label')

    for i in range(num_file):
        ax[i, 0].imshow(np.transpose(test_image_np_org_s[i], (1, 2, 0))[:, :, :3])
        ax[i, 1].imshow(hr_label_s_rgb[i])
        ax[i, 2].imshow(test_preds_np_rgb[i])
        ax[i, 3].imshow(nlcd_np_s_rgb[i])

    os.makedirs(OUTPUT_IMAGE_DIRECTORY, exist_ok = True)
    output_name = OUTPUT_IMAGE_DIRECTORY + OUTPUT_FILES_NAME
    if EVALUATE_ENSEMBLE_FLAG:
        output_name = output_name + '_ENS_EVAL_M' + str(NUM_ENSEMBLE)
        if ex_flag:
            output_name = output_name + '_ex'
            print('one_example_model')
        else:
            print('ensemble_model')
    if chain_flag:
        output_name = output_name + '_model_' + str(chain_num)

    fig.savefig(output_name +'_aligned_preds.png', dpi = fig.dpi)
    plt.show()

    if EVALUATE_ENSEMBLE_FLAG:
        print('\nplot considering uncertainty:\n', titles)
        fig, ax = plt.subplots(num_file, 4, figsize = (20, 10 * int(num_file / 2)))
        ax[0, 0].set_title('NAIP image')
        ax[0, 1].set_title('Ground truth')
        ax[0, 2].set_title('Prediction')
        ax[0, 3].set_title('NLCD label')

        for i in range(test_preds_np_rgb.shape[3]):
            test_preds_np_rgb[:, :, :, i] = np.clip(test_preds_np_rgb[:, :, :, i] + 255 * uncertainty_s, 0, 255)
        test_preds_np_rgb = test_preds_np_rgb.astype(np.uint8)
        for i in range(num_file):
            ax[i, 0].imshow(np.transpose(test_image_np_org_s[i], (1, 2, 0))[:, :, :3])
            ax[i, 1].imshow(hr_label_s_rgb[i])
            ax[i, 2].imshow(test_preds_np_rgb[i])
            ax[i, 3].imshow(nlcd_np_s_rgb[i])

        if ex_flag:
            print('one_example_model')
        else:
            print('ensemble_model')

        fig.savefig(output_name +'_aligned_preds_unc.png', dpi = fig.dpi)
        plt.show()

    return(titles, num_file, hr_label_s_rgb, test_preds_np_rgb, nlcd_np_s, nlcd_np_s_rgb)

def plot_save_log(losses, test_losses, IoU, test_IoU, output_name, none_flag = True):
    output_path = OUTPUT_IMAGE_DIRECTORY + output_name

    fig = plt.figure(figsize = (16, 4))
    plt.plot(losses, label = 'train')
    plt.plot(test_losses, label = 'test')
    plt.xlabel('# Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss plot')
    fig.savefig(output_path +'_loss.png', dpi = fig.dpi)
    plt.show()

    all_class_names = ['Water', 'Tree Canopy', 'Low Vegetation', 'Impervious', 'None']
    if not(none_flag):
        IoU = IoU[:-1]
        test_IoU = test_IoU[:-1]
        all_class_names = all_class_names[:-1]
        output_path = output_path + '_wo_None'

    fig = plt.figure(figsize = (16, 4))
    for i, ious in enumerate(IoU):
        ious = pd.Series(ious).rolling(1).mean()
        plt.plot(ious, label = all_class_names[i])

    plt.legend()
    plt.xlabel('# Steps')
    plt.ylabel('Train IoU')
    plt.title('Train IoU plot (smoothed)')
    fig.savefig(output_path + '_Train_IoU.png', dpi = fig.dpi)
    plt.show()

    fig = plt.figure(figsize = (16, 4))
    for i, ious in enumerate(test_IoU):
        plt.plot(ious, label = all_class_names[i])
    plt.legend()
    plt.xlabel('# Steps')
    plt.ylabel('Test IoU')
    plt.title('Test IoU plot')
    fig.savefig(output_path + '_Test_IoU.png', dpi = fig.dpi)
    plt.show()

def save_plot_outputs(losses, test_losses, IoU, test_IoU, test_preds, test_image_np_s, test_image_np_org_s,
                      test_label_np_s, chain_flag = False, chain_cont_flag = False, chain_num = 1):
    if chain_flag:
        _ = visualize_save_pred(test_preds, test_image_np_org_s, chain_flag, chain_num)
        output_name = OUTPUT_FILES_NAME + '_model_' + str(chain_num)
    else:
        output_name = OUTPUT_FILES_NAME

    os.makedirs(TRAINED_OBJECT_DIRECTORY, exist_ok = True)
    torch.save(model.state_dict(), TRAINED_OBJECT_DIRECTORY + 'trained_model_' + output_name + '.pth')
    torch.save(optimizer.state_dict(), TRAINED_OBJECT_DIRECTORY + 'trained_optimizer_' + output_name + '.pth')
    trained_objects = losses, test_losses, IoU, test_IoU, test_preds, test_image_np_s, test_image_np_org_s, test_label_np_s
    pd.to_pickle(trained_objects, TRAINED_OBJECT_DIRECTORY + 'trained_objects_' + output_name + '.pkl')

    plot_save_log(losses, test_losses, IoU, test_IoU, output_name, none_flag = True)
    plot_save_log(losses, test_losses, IoU, test_IoU, output_name, none_flag = False)

    if chain_cont_flag:
        del globals()['model'], globals()['losses'], globals()['test_losses'], globals()['IoU'], globals()['test_IoU']
        del globals()['test_preds'], globals()['test_image_np_s'], globals()['test_image_np_org_s'], globals()['test_label_np_s']
        gc.collect()

In [ ]:
def calc_preds_unc(models, image_path, image_loader, codalab_flag = False):
    from scipy.stats import entropy

    test_data, test_image_np_s, test_image_np_org_s, pad_width = \
            make_image_data(image_path = image_path, image_loader = image_loader)
    probs_models = np.empty((0, len(image_loader), NUM_DFC2021_CLASS, 3880, 3880))
    for model in models:
        probs_s = np.empty((0, NUM_DFC2021_CLASS, SPL_WH, SPL_WH))
        for i in range(int(test_data.shape[0] / BATCH)):
            outputs = model(test_data[i * BATCH:(i + 1) * BATCH])
            probs = nn.Softmax(dim = 1)(outputs)
            probs_s = np.append(probs_s, probs.to('cpu').detach().numpy().copy(), 0)
        if test_data.shape[0] % BATCH > 0:
            outputs = model(test_data[(i + 1) * BATCH:])
            probs = nn.Softmax(dim = 1)(outputs)
            probs_s = np.append(probs_s, probs.to('cpu').detach().numpy().copy(), 0)

        reshaped_probs_s = np.empty((len(image_loader), 0, 3880, 3880))
        for i in range(NUM_DFC2021_CLASS):
            reshaped_probs_s = np.append(reshaped_probs_s, reshape_data(probs_s[:, i, :, :]).reshape([-1, 1, 3880, 3880]), 1)
        assert reshaped_probs_s.shape == (len(image_loader), NUM_DFC2021_CLASS, 3880, 3880)
        probs_models = np.append(probs_models, reshaped_probs_s.reshape([1, len(image_loader), NUM_DFC2021_CLASS, 3880, 3880]), 0)

    probs_ens = probs_models.mean(axis = 0)
    preds_ens = np.argmax(probs_ens, axis = 1)
    if codalab_flag:
        uncertainty_ens = []
        preds_model_ex = []
        uncertainty_model_ex = []
    else:
        uncertainty_ens = entropy(probs_ens, axis = 1) / np.log(NUM_DFC2021_CLASS)
        preds_model_ex = np.argmax(probs_models[0], axis = 1)
        uncertainty_model_ex = entropy(probs_models[0], axis = 1) / np.log(NUM_DFC2021_CLASS)

    return(preds_ens, uncertainty_ens, preds_model_ex, uncertainty_model_ex, test_image_np_org_s, pad_width)

## Run Training and Visualizing Codes

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)        #GPU: cuda:0, CPU: cpu

chain_flag = True if len(MODEL_LIST) > 1 else False

if EVALUATE_ENSEMBLE_FLAG:
    model_name = MODEL_LIST[-1]
    models = []
    for i in range(NUM_ENSEMBLE):
        if model_name == 'FCN1l':
            model = FCN1l(num_input_channels = 4, num_output_classes = NUM_DFC2021_CLASS).to(device)
        elif model_name == 'FCN5l':
            model = FCN5l(num_input_channels = 4, num_output_classes = NUM_DFC2021_CLASS).to(device)
        elif model_name == 'U-NET18':
            model = smp.Unet(encoder_name = 'resnet18', encoder_depth = 3, encoder_weights = None, decoder_channels = (128, 64, 64),
                            decoder_use_batchnorm = True, in_channels = 4, classes = NUM_DFC2021_CLASS).to(device)
        elif model_name == 'U-NET50':
            model = smp.Unet(encoder_name = 'resnet50', encoder_depth = 3, encoder_weights = None, decoder_channels = (128, 64, 64),
                            decoder_use_batchnorm = True, in_channels = 4, classes = NUM_DFC2021_CLASS).to(device)
        
        output_file_path = TRAINED_OBJECT_DIRECTORY + 'trained_model_' + OUTPUT_FILES_NAME + '_ENS_' + str(i)
        output_file_path = output_file_path + ('_model_' + str(len(MODEL_LIST)) + '.pth' if chain_flag else '.pth')
        model.load_state_dict(torch.load(output_file_path))
        models.append(model)

In [ ]:
if EVALUATE_ENSEMBLE_FLAG:
    start_time = time.time()
    test_image_loader = sorted([file for file in os.listdir(TEST_NAIP_DIRECTORY) if file.endswith('.tif')])

    preds_ens_s = np.empty((0, 3880, 3880))
    uncertainty_ens_s = np.empty((0, 3880, 3880))
    preds_model_ex_s = np.empty((0, 3880, 3880))
    uncertainty_model_ex_s = np.empty((0, 3880, 3880))
    test_image_np_org_s = np.empty((0, 4, 3880, 3880))
    for ite in tqdm(range(int(len(test_image_loader) / BATCH))):
        preds_ens, uncertainty_ens, preds_model_ex, uncertainty_model_ex, test_image_np_org, _ = \
                 calc_preds_unc(models = models, image_path = TEST_NAIP_DIRECTORY,
                                image_loader = test_image_loader[(ite * BATCH):((ite + 1) * BATCH)])
        preds_ens_s = np.append(preds_ens_s, preds_ens, 0)
        uncertainty_ens_s = np.append(uncertainty_ens_s, uncertainty_ens, 0)
        preds_model_ex_s = np.append(preds_model_ex_s, preds_model_ex, 0)
        uncertainty_model_ex_s = np.append(uncertainty_model_ex_s, uncertainty_model_ex, 0)
        test_image_np_org_s = np.append(test_image_np_org_s, test_image_np_org, 0)

    if len(test_image_loader) % BATCH > 0:
        preds_ens, uncertainty_ens, preds_model_ex, uncertainty_model_ex, test_image_np_org, _ = \
                calc_preds_unc(models = models, image_path = TEST_NAIP_DIRECTORY,
                               image_loader = test_image_loader[((ite + 1) * BATCH):])
        preds_ens_s = np.append(preds_ens_s, preds_ens, 0)
        uncertainty_ens_s = np.append(uncertainty_ens_s, uncertainty_ens, 0)
        preds_model_ex_s = np.append(preds_model_ex_s, preds_model_ex, 0)
        uncertainty_model_ex_s = np.append(uncertainty_model_ex_s, uncertainty_model_ex, 0)
        test_image_np_org_s = np.append(test_image_np_org_s, test_image_np_org, 0)

    print("\nmemory_usage_after_calc_test_preds_uncertainty")
    print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

    time_lapsed = time.time() - start_time
    print(f'\n{time_lapsed // 60} min {int(time_lapsed % 60)} sec lapsed')

else:
    torch.manual_seed(SEED_NUM)
    random.seed(SEED_NUM)
    np.random.seed(SEED_NUM)

    for i, model_name in enumerate(MODEL_LIST):
        if model_name == 'FCN1l':
            model = FCN1l(num_input_channels = 4, num_output_classes = NUM_DFC2021_CLASS).to(device)
        elif model_name == 'FCN5l':
            model = FCN5l(num_input_channels = 4, num_output_classes = NUM_DFC2021_CLASS).to(device)
        elif model_name == 'U-NET18':
            model = smp.Unet(encoder_name = 'resnet18', encoder_depth = 3, encoder_weights = None, decoder_channels = (128, 64, 64),
                            decoder_use_batchnorm = True, in_channels = 4, classes = NUM_DFC2021_CLASS).to(device)
        elif model_name == 'U-NET50':
            model = smp.Unet(encoder_name = 'resnet50', encoder_depth = 3, encoder_weights = None, decoder_channels = (128, 64, 64),
                            decoder_use_batchnorm = True, in_channels = 4, classes = NUM_DFC2021_CLASS).to(device)

        optimizer = torch.optim.RAdam(model.parameters(), lr = 0.001) #torch.optim.NAdam(model.parameters(), lr = 0.001), torch.optim.Adam(model.parameters(), lr = 0.001)

        chain_cont_flag = True if i + 1 < len(MODEL_LIST) else False
        model, losses, test_losses, IoU, test_IoU, test_preds, test_image_np_s, test_image_np_org_s, test_label_np_s = \
                    training(model, optimizer, epoch = EPOCH, bsize = BATCH, ite_show_log = 64, chain_flag = chain_flag, chain_num = i + 1,
                            losses = [], test_losses = [], l_IoU = [[]], l_test_IoU = [[]])

        if chain_cont_flag:
            train_image_loader = sorted([file for file in os.listdir(TRAIN_NAIP_DIRECTORY) if file.endswith('.tif')])
            chain_n_directory = CHAIN_OUTPUT_DIRECTORY + OUTPUT_FILES_NAME + '_model_' + str(i + 1) + '/'
            make_save_preds(model = model, bsize = BATCH, image_loader = train_image_loader,
                            naip_dir = TRAIN_NAIP_DIRECTORY, save_pred_dir = chain_n_directory, codalab_flag = True)

        save_plot_outputs(losses, test_losses, IoU, test_IoU, test_preds, test_image_np_s, test_image_np_org_s,
                        test_label_np_s, chain_flag = chain_flag, chain_cont_flag = chain_cont_flag, chain_num = i + 1)

# Visualization of Results

## Aligned images

In [ ]:
if EVALUATE_ENSEMBLE_FLAG:
    titles_ex, num_file_ex, hr_label_s_rgb_ex, test_preds_np_rgb_ex, nlcd_np_s_ex, nlcd_np_s_rgb_ex = \
                    visualize_save_pred(preds_model_ex_s, test_image_np_org_s, chain_flag, chain_num = len(MODEL_LIST),
                                        uncertainty_s = uncertainty_model_ex_s, ex_flag = True)
    titles, num_file, hr_label_s_rgb, test_preds_np_rgb, nlcd_np_s, nlcd_np_s_rgb = \
                    visualize_save_pred(preds_ens_s, test_image_np_org_s, chain_flag, chain_num = len(MODEL_LIST),
                                        uncertainty_s = uncertainty_ens_s, ex_flag = False)
else:
    titles, num_file, hr_label_s_rgb, test_preds_np_rgb, nlcd_np_s, nlcd_np_s_rgb = \
                    visualize_save_pred(test_preds, test_image_np_org_s, chain_flag, chain_num = len(MODEL_LIST))

#IoU for Gain and Loss

In [ ]:
def calc_GL_IoU(both_year_preds_np, both_year_true_labels_np):
    by_preds_tensor = torch.tensor(both_year_preds_np + 1).reshape(-1, 2, 3880, 3880)
    by_true_labels_tensor = torch.tensor(both_year_true_labels_np + 1).reshape(-1, 2, 3880, 3880)
    preds_change_flag = by_preds_tensor[:, 0] != by_preds_tensor[:, 1]
    true_change_flag = by_true_labels_tensor[:, 0] != by_true_labels_tensor[:, 1]

    by_preds_tensor[:, 0] = by_preds_tensor[:, 0] * preds_change_flag
    by_preds_tensor[:, 1] = by_preds_tensor[:, 1] * preds_change_flag
    by_true_labels_tensor[:, 0] = by_true_labels_tensor[:, 0] * true_change_flag
    
    by_true_labels_tensor[:, 1] = by_true_labels_tensor[:, 1] * true_change_flag

    print("preds_change_flag:\n", pd.Series(preds_change_flag.flatten()).value_counts())
    print("\ntrue_change_flag:\n", pd.Series(true_change_flag.flatten()).value_counts())

    print("\n\n0: No Change, 1: -Water, 2: -Tree Canopy, 3: -Low Vegetation, 4: -Impervious, 5: -None",
          "\npred_label_2013:\n", pd.Series(by_preds_tensor[:, 0].flatten()).value_counts(),
          "\n\ntest_label_2013:\n", pd.Series(by_true_labels_tensor[:, 0].flatten()).value_counts())

    print("\n\n0: No Change, 1: +Water, 2: +Tree Canopy, 3: +Low Vegetation, 4: +Impervious, 5: +None",
          "\npred_label_2017:\n", pd.Series(by_preds_tensor[:, 1].flatten()).value_counts(),
          "\n\ntest_label_2017:\n", pd.Series(by_true_labels_tensor[:, 1].flatten()).value_counts())

    IoU = []
    for i in range(NUM_DFC2021_CLASS - 1):
        IoU.append(calc_IoU(by_preds_tensor[:, 0], by_true_labels_tensor[:, 0], i + 1).item())
    for i in range(NUM_DFC2021_CLASS - 1):
        IoU.append(calc_IoU(by_preds_tensor[:, 1], by_true_labels_tensor[:, 1], i + 1).item())
    IoU.append(np.mean(IoU))

    print(f"\n\nIoU: [-W, -TC, -LV, -I, +W, +TC, +LV, +I, Avg.] = {np.round(IoU, 3)}")

In [ ]:
if EVALUATE_ENSEMBLE_FLAG:
    _, test_label_np_s = make_test_targets(label_path = TEST_HR_NP_PNG_PATH)
    print('one_example_model:\n')
    calc_GL_IoU(preds_model_ex_s, test_label_np_s)
    print('\n\nensemble_model:\n')
    calc_GL_IoU(preds_ens_s, test_label_np_s)
else:
    test_preds_ch = test_preds.to('cpu').detach().numpy().copy()
    calc_GL_IoU(test_preds_ch, test_label_np_s.cpu())

## IoU for Gain and Loss by NLCD Difference Algorithm 


In [ ]:
if EVALUATE_ENSEMBLE_FLAG:
    calc_GL_IoU(np.array(nlcd_np_s), test_label_np_s)
else:
    calc_GL_IoU(np.array(nlcd_np_s), test_label_np_s.cpu())

# Make a zip file for submission to [Codalab](https://codalab.lisn.upsaclay.fr/competitions/7908)

Codalab can then compute the test IoU(s) in exactly the same way as the DFC2021 contest.


In [ ]:
# Use the git implementation of the 3rd place team of the DFC 2021 contest (as base-line model git does not include test_tiles.txt used below).
!git clone https://github.com/baoqianyue/DFC2021-Track-MSD.git

In [ ]:
df = pd.read_csv(ROOT_PATH + 'DFC2021-Track-MSD/data/test_tiles.txt', header = None)
df = df.rename(columns = {0:'id'})

def id_to_url_2013(id):
    url = 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/' + str(id) + '_naip-2013.tif'
    return url

def id_to_url_2017(id):
    url = 'https://dfc2021.blob.core.windows.net/competition-data/naip-2017/' + str(id) + '_naip-2017.tif'
    return url

df['url_2013'] = df['id'].apply(id_to_url_2013)
df['url_2017'] = df['id'].apply(id_to_url_2017)

In [ ]:
# Need to run the codes below once when running codes in this notebook first by setting CODALAB_NAIP_DOWNLOAD_FLAG = True.

if CODALAB_NAIP_DOWNLOAD_FLAG:
    start_time = time.time()
    shutil.rmtree(CODALAB_NAIP_DIRECTORY, ignore_errors = True)
    for url in df['url_2013']:
        download_file(save_path = CODALAB_NAIP_DIRECTORY, save_file_name = url[65:], url = url)
    for url in df['url_2017']:
        download_file(save_path = CODALAB_NAIP_DIRECTORY, save_file_name = url[65:], url = url)
    print((time.time() - start_time) / 60)

In [ ]:
codalab_test_image_loader = sorted([file for file in os.listdir(CODALAB_NAIP_DIRECTORY) if file.endswith('.tif')])

if EVALUATE_ENSEMBLE_FLAG:
    start_time = time.time()

    shutil.rmtree(CODALAB_PRED_DIRECTORY, ignore_errors = True)
    os.makedirs(CODALAB_PRED_DIRECTORY, exist_ok = True)

    for ite in tqdm(range(int(len(codalab_test_image_loader) / BATCH))):
        image_loader = codalab_test_image_loader[(ite * BATCH):((ite + 1) * BATCH)]
        preds_ens, _, _, _, _, pad_width = calc_preds_unc(models = models, image_path = CODALAB_NAIP_DIRECTORY,
                                                          image_loader = image_loader, codalab_flag = True)
        # Class "None" -> modified to "Tree Canopy" (because TC is most prevalent)
        preds_ens[preds_ens == 4] = 1

        for n in range(BATCH):
            reshape_and_save_tif(preds = preds_ens[n], pw_x_bef = pad_width['pw_x_bef'][n],
                                 pw_x_aft = pad_width['pw_x_aft'][n], pw_y_bef = pad_width['pw_y_bef'][n],
                                 pw_y_aft = pad_width['pw_y_aft'][n], file = image_loader[n],
                                 naip_directory = CODALAB_NAIP_DIRECTORY, save_directory = CODALAB_PRED_DIRECTORY)
            
    if len(codalab_test_image_loader) % BATCH > 0:
        image_loader = codalab_test_image_loader[((ite + 1) * BATCH):]
        preds_ens, _, _, _, _, pad_width = calc_preds_unc(models = models, image_path = CODALAB_NAIP_DIRECTORY,
                                                          image_loader = image_loader, codalab_flag = True)
        # Class "None" -> modified to "Tree Canopy" (because TC is most prevalent)
        preds_ens[preds_ens == 4] = 1

        for n in range(len(codalab_test_image_loader) % BATCH):
            reshape_and_save_tif(preds = preds_ens[n], pw_x_bef = pad_width['pw_x_bef'][n],
                                 pw_x_aft = pad_width['pw_x_aft'][n], pw_y_bef = pad_width['pw_y_bef'][n],
                                 pw_y_aft = pad_width['pw_y_aft'][n], file = image_loader[n],
                                 naip_directory = CODALAB_NAIP_DIRECTORY, save_directory = CODALAB_PRED_DIRECTORY)

    print("\nmemory_usage_after_calc_codalab_preds")
    print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

    time_lapsed = time.time() - start_time
    print(f'\n{time_lapsed // 60} min {int(time_lapsed % 60)} sec lapsed')

else:
    make_save_preds(model = model, bsize = BATCH, image_loader = codalab_test_image_loader,
                    naip_dir = CODALAB_NAIP_DIRECTORY, save_pred_dir = CODALAB_PRED_DIRECTORY, codalab_flag = True)

In [ ]:
##########################################################################################
### Adapt from independent_pairs_to_predictions.py in the git of baseline or 3rd-place team
##########################################################################################

shutil.rmtree(CODALAB_SUBMISSION_DIRECTORY, ignore_errors = True)
%cd /content/DFC2021-Track-MSD


parser = argparse.ArgumentParser(description = 'Helper script for combining DFC2021 prediction into submission format')
parser.add_argument('--input_dir', type = str, required = True,
                    help = 'The path to a directory containing the output of the `inference.py` script.')
parser.add_argument('--output_dir', type = str, required = True,
                    help = 'The path to output the consolidated predictions, should be different than `--input_dir`.')
parser.add_argument('--overwrite', action = "store_true", help = 'Flag for overwriting `--output_dir` if that directory already exists.')
parser.add_argument('--soft_assignment', action = "store_true",
                    help = 'Flag for combining predictions using soft assignment. You can only use this if you ran the `inference.py` script with the `--save_soft` flag.')
args = parser.parse_args(args=['--input_dir', CODALAB_PRED_DIRECTORY, '--output_dir', CODALAB_SUBMISSION_DIRECTORY])


def main():
    print("Starting to combine predictions at %s" % (str(datetime.datetime.now())))

    #-------------------
    # Setup
    #-------------------
    assert os.path.exists(args.input_dir) and len(os.listdir(args.input_dir)) > 0

    if os.path.isfile(args.output_dir):
        print("A file was passed as `--output_dir`, please pass a directory!")
        return

    if os.path.exists(args.output_dir) and len(os.listdir(args.output_dir)) > 0:
        if args.overwrite:
            print("WARNING! The output directory, %s, already exists, we might overwrite data in it!" % (args.output_dir))
        else:
            print("The output directory, %s, already exists and isn't empty. We don't want to overwrite and existing results, exiting..." % (args.output_dir))
            return
    else:
        print("The output directory doesn't exist or is empty.")
        os.makedirs(args.output_dir, exist_ok=True)


    #-------------------
    # Run for each pair of predictions that we find in `--input_dir`
    #-------------------
    idxs_2013 = [
        fn.split("_")[0]
        for fn in os.listdir(args.input_dir)
        if fn.endswith("predictions-2013.tif")
    ]

    idxs_2017 = [
        fn.split("_")[0]
        for fn in os.listdir(args.input_dir)
        if fn.endswith("predictions-2017.tif")
    ]

    assert len(idxs_2013) > 0, "No matching files found in '%s'" % (args.input_dir)
    assert set(idxs_2013) == set(idxs_2017), "Missing some predictions"

    for i, idx in enumerate(idxs_2013):
        tic = time.time()

        print("(%d/%d) Processing tile %s" % (i, len(idxs_2013), idx), end = " ... ")

        if args.soft_assignment:
            fn_2013 = os.path.join(args.input_dir, "%s_predictions-soft-2013.tif" % (idx))
            fn_2017 = os.path.join(args.input_dir, "%s_predictions-soft-2017.tif" % (idx))
        else:
            fn_2013 = os.path.join(args.input_dir, "%s_predictions-2013.tif" % (idx))
            fn_2017 = os.path.join(args.input_dir, "%s_predictions-2017.tif" % (idx))
        output_fn = os.path.join(args.output_dir, "%s_predictions.tif" % (idx))

        assert os.path.exists(fn_2013) and os.path.exists(fn_2017)

        ## Load the independent predictions for both years
        with rasterio.open(fn_2013) as f:
            if args.soft_assignment:
                t1 = np.rollaxis(f.read(), 0, 3)
            else:
                t1 = f.read(1)
            input_profile = f.profile.copy() # save the metadata for writing output
            
        with rasterio.open(fn_2017) as f:
            if args.soft_assignment:
                t2 = np.rollaxis(f.read(), 0, 3)
            else:
                t2 = f.read(1)

        t1_reduced = t1  # changed from the original git implementation
        t2_reduced = t2  # changed from the original git implementation

        ## Convert the two layers of predictions into the format expected by codalab
        predictions = (t1_reduced * 4) + t2_reduced
        predictions[predictions == 5] = 0
        predictions[predictions == 10] = 0
        predictions[predictions == 15] = 0
        predictions = predictions.astype(np.uint8)

        ## Write output as GeoTIFF
        input_profile["count"] = 1
        with rasterio.open(output_fn, "w", **input_profile) as f:
            f.write(predictions, 1)

        print("finished in %0.4f seconds" % (time.time() - tic))

if __name__ == "__main__":
    main()

You can sign up to Codalab and submit the zip file after downloading it.


In [ ]:
os.makedirs(CODALAB_SUBMISSION_ZIP_DIRECTORY, exist_ok = True)
os.chdir(CODALAB_SUBMISSION_ZIP_DIRECTORY)
if EVALUATE_ENSEMBLE_FLAG:
    shutil.make_archive(OUTPUT_FILES_NAME + '_ENS_EVAL_M' + str(NUM_ENSEMBLE), format = 'zip', root_dir = CODALAB_SUBMISSION_DIRECTORY)
else:
    shutil.make_archive(OUTPUT_FILES_NAME, format = 'zip', root_dir = CODALAB_SUBMISSION_DIRECTORY)

## Check the output

In [ ]:
codalab_pred_loader = sorted([file for file in os.listdir(CODALAB_SUBMISSION_DIRECTORY) if file.endswith('.tif')])

for i, file in enumerate(tqdm(codalab_pred_loader)):
    ds = gdal.Open(os.path.join(CODALAB_SUBMISSION_DIRECTORY, file), gdal.GA_ReadOnly)
    codalab_pred_np = np.array([ds.GetRasterBand(i + 1).ReadAsArray() for i in range(ds.RasterCount)])
    if max(codalab_pred_np[0].flatten()) > 14:
        print(f'\n\n{file} is wrong!')
        print(f'\nsize: ({ds.RasterXSize}, {ds.RasterYSize})\n')
        print(pd.Series(codalab_pred_np[0].flatten()).value_counts())
        break
    if i == len(codalab_pred_loader) - 1:
        print('\n\nThe format looks OK')